In [198]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl.query import Q
from fuzzywuzzy import fuzz
import re

In [199]:
books = [book.lower() for book in open('../files/books.txt').read().split('\n')]

In [200]:
client = Elasticsearch()
s = Search(using=client)

In [228]:
verse_parser = re.compile(r'(\d\s)?([\w\.]+)\s+([\d:,\-\s\;]+)')
def search(term):
    match = verse_parser.match(term)
    if match:
        response = s.query('match', verse=term).execute()
        for hit in response:
            print(hit.verse)
        return
    book_match = max(list(map(lambda book: fuzz.ratio(term.strip().lower(), book), books)))
    sorted_books = list(reversed(sorted(books, key=lambda book: fuzz.ratio(term.strip().lower(), book))))
    if book_match >= 60:
        book_query = Q('bool', must=[Q('match', book=sorted_books[0])])
        response = s.query(book_query).execute()
        for hit in response:
            print(hit.verse)
        return
    else:
        query_string = Q(
            'match_phrase', text={'query': term, 'slop': 2})
        response = s.query(query_string).execute()
        for hit in response:
            print((hit.verse, hit.text))
        return

In [229]:
search('joh 1:1')

1 Samuel 1:1
1 Thessalonians 1:1
1 Peter 1:1
1 Corinthians 1:1
1 Kings 1:1
1 Timothy 1:1
1 John 1:1
1 Chronicles 1:1
Deuteronomy 1:1
Genesis 1:1
